In [118]:
import pandas as pd
import numpy as np
import json
import gzip
import os
import concurrent.futures
import glob
import io
import seaborn as sns
import multiprocessing
from tqdm import tqdm
import mmap
from multiprocessing import Pool
from os import listdir

In [119]:
path = '/work-ceph/bizer-tp2021/data_integration_using_deep_learning/src/data/data/CSV_files'

In [120]:
files = pd.read_csv(path + '/all_files_cleaned.csv')
files = files.drop(columns={'Unnamed: 0'})
files

,schema_type,column_name,data_type,percentage,filename,num_tables,prob_matches
0,name/ Product_stats.csv,name,string,0.994361,Product_stats.csv,432728.0,product_name
1,offers/ Product_stats.csv,offers,string,0.964249,Product_stats.csv,419624.0,offer
2,brand/ Product_stats.csv,brand,string,0.403728,Product_stats.csv,175695.0,product_name
3,headline/ CreativeWork_stats.csv,headline,string,0.939871,CreativeWork_stats.csv,76451.0,headline
4,author/ CreativeWork_stats.csv,author,string,0.776499,CreativeWork_stats.csv,63162.0,person_name
...,...,...,...,...,...,...,...
202,numtracks/ MusicAlbum_stats.csv,numtracks,int,0.415808,MusicAlbum_stats.csv,121.0,number
203,menu/ Restaurant_stats.csv,menu,string,0.104581,Restaurant_stats.csv,121.0,text_description
204,contactpoint/ Person_stats.csv,contactpoint,string,0.009353,Person_stats.csv,120.0,text_description
205,email/ Hotel_stats.csv,email,string,0.107664,Hotel_stats.csv,118.0,email


In [121]:
columns_sorted = files.groupby('filename')['column_name'].apply(list).reset_index(name='listofcolumns')
columns_sorted.filename = columns_sorted.filename.str.split('_', expand = True)
columns_sorted.index = columns_sorted['filename']
columns_sorted

,filename,listofcolumns
filename,,
Book,Book,"[name, author, isbn, publisher, offers, datepu..."
CreativeWork,CreativeWork,"[headline, author, datepublished, datemodified..."
Event,Event,"[name, enddate, location, performer, organizer..."
Hotel,Hotel,"[name, address, telephone, pricerange, geo, ag..."
LocalBusiness,LocalBusiness,"[name, address, telephone, geo, pricerange, ag..."
MusicAlbum,MusicAlbum,"[name, byartist, track, numtracks]"
MusicRecording,MusicRecording,"[name, duration, byartist, inalbum, datepublis..."
Person,Person,"[mainentityofpage, jobtitle, birthdate, givenn..."
Place,Place,"[name, geo, telephone, sameas, openinghoursspe..."


In [239]:
inputdir = '../../src/data/schemafiltereddata/Book/'
files = [file for file in os.listdir(inputdir) if file.endswith('.json.gz')]
books = pd.DataFrame ()
setcol = set(columns_sorted.loc['Book']['listofcolumns'])
for file in files:
    df = pd.read_json(os.path.join(inputdir+file), compression = 'gzip', lines=True)
    numrows = df.shape[0]
    numcolumns = df.shape[1]
    numselcol = df[df.columns[df.columns.isin(setcol)]].shape[1]
    list_numNAs = df[df.columns[df.columns.isin(setcol)]].isnull().sum().sum()
    list_col = list(df.columns[df.columns.isin(setcol)])
    books = books.append({'Class':'Book','filename':file,'num_rows':numrows, 'num_columns': numcolumns, 'num_columns_selected':numselcol, 'numNAs': list_numNAs, 'columns':list_col}, ignore_index=True)
#books = books.explode(['columns', 'numNAs'])
#books = books.explode('numNAs')
#books = books[books['columns'].isin(setcol)]

display(books)

,Class,columns,filename,numNAs,num_columns,num_columns_selected,num_rows
0,Book,"[name, author, publisher, offers, bookedition]",Book_foldvaribooks.com_September2020.json.gz,336.0,8.0,5.0,274.0
1,Book,"[name, author, publisher, isbn, datepublished,...",Book_virago.co.uk_September2020.json.gz,3.0,11.0,6.0,398.0
2,Book,"[name, author, publisher, offers, bookedition]",Book_borgantiquarian.com_September2020.json.gz,316.0,8.0,5.0,248.0
3,Book,"[name, author, publisher, isbn, datepublished]",Book_downpour.com_September2020.json.gz,7358.0,8.0,5.0,12194.0
4,Book,"[name, author, publisher, isbn, datepublished,...",Book_marasworld.com_September2020.json.gz,0.0,8.0,6.0,26.0
...,...,...,...,...,...,...,...
973,Book,"[name, publisher, isbn, bookformat, datepublis...",Book_benjamins.com_September2020.json.gz,2363.0,14.0,8.0,1795.0
974,Book,"[name, author, publisher, isbn, datepublished,...",Book_nollegiu.com_September2020.json.gz,2139.0,10.0,7.0,3175.0
975,Book,"[name, author, publisher, isbn, datepublished,...",Book_nosegraze.com_September2020.json.gz,111.0,9.0,7.0,153.0
976,Book,"[name, author, publisher, isbn, datepublished,...",Book_hachettebooks.com_September2020.json.gz,46.0,12.0,7.0,583.0


In [ ]:
#inputdir = '../../src/data/schemafiltereddata/CreativeWork/'
files = [file for file in os.listdir(inputdir) if file.endswith('.json.gz')]
cw = pd.DataFrame ()
setcol = set(columns_sorted.loc['CreativeWork']['listofcolumns'])
for file in files:
    df = pd.read_json(os.path.join(inputdir+file), compression = 'gzip', lines=True)
    numrows = df.shape[0]
    numcolumns = df.shape[1]
    numselcol = df[df.columns[df.columns.isin(setcol)]].shape[1]
    list_numNAs = df[df.columns[df.columns.isin(setcol)]].isnull().sum().sum()
    list_col = list(df.columns)
    cw = cw.append({'Class':'CreativeWork','filename':file,'num_rows':numrows, 'num_columns': numcolumns, 'num_columns_selected':numselcol, 'numNAs': list_numNAs, 'columns':list_col}, ignore_index=True)
cw = cw.explode('columns')
#books = books.explode('numNAs')
cw = cw[cw['columns'].isin(setcol)]

display(cw)

In [ ]:
print(cw[cw['num_rows']<=10])

In [123]:
inputdir = '../../src/data/schemafiltereddata/Book/'
files = [file for file in os.listdir(inputdir) if file.endswith('.json.gz')]
books = pd.DataFrame ()
setcol = set(columns_sorted.loc['Book']['listofcolumns'])
for file in files:
    df = pd.read_json(os.path.join(inputdir+file), compression = 'gzip', lines=True)
    numrows = df.shape[0]
    numcolumns = df.shape[1]
    numselcol = df[df.columns[df.columns.isin(setcol)]].shape[1]
    numNAs = list(df[df.columns[df.columns.isin(setcol)]].isnull().sum())
    list_col = list(df.columns[df.columns.isin(setcol)])
    books = books.append({'Class':'Book','filename':file,'num_rows':numrows, 'num_columns': numcolumns, 'num_columns_selected':numselcol, 'numNAs': numNAs, 'columns':list_col}, ignore_index=True)
#books = books.explode('columns')
#books = books[books['columns'].isin(setcol)]
display(books)

,Class,columns,filename,numNAs,num_columns,num_columns_selected,num_rows
0,Book,"[name, author, publisher, offers, bookedition]",Book_foldvaribooks.com_September2020.json.gz,"[1, 40, 157, 0, 138]",8.0,5.0,274.0
1,Book,"[name, author, publisher, isbn, datepublished,...",Book_virago.co.uk_September2020.json.gz,"[0, 2, 0, 0, 0, 1]",11.0,6.0,398.0
2,Book,"[name, author, publisher, offers, bookedition]",Book_borgantiquarian.com_September2020.json.gz,"[0, 0, 157, 0, 159]",8.0,5.0,248.0
3,Book,"[name, author, publisher, isbn, datepublished]",Book_downpour.com_September2020.json.gz,"[0, 13, 0, 7323, 22]",8.0,5.0,12194.0
4,Book,"[name, author, publisher, isbn, datepublished,...",Book_marasworld.com_September2020.json.gz,"[0, 0, 0, 0, 0, 0]",8.0,6.0,26.0
...,...,...,...,...,...,...,...
973,Book,"[name, publisher, isbn, bookformat, datepublis...",Book_benjamins.com_September2020.json.gz,"[0, 7, 756, 755, 62, 775, 8, 0]",14.0,8.0,1795.0
974,Book,"[name, author, publisher, isbn, datepublished,...",Book_nollegiu.com_September2020.json.gz,"[0, 77, 0, 36, 1465, 0, 561]",10.0,7.0,3175.0
975,Book,"[name, author, publisher, isbn, datepublished,...",Book_nosegraze.com_September2020.json.gz,"[0, 0, 0, 109, 0, 1, 1]",9.0,7.0,153.0
976,Book,"[name, author, publisher, isbn, datepublished,...",Book_hachettebooks.com_September2020.json.gz,"[0, 0, 0, 0, 0, 45, 1]",12.0,7.0,583.0


In [ ]:
allRows_selCol = books.num_columns*books.num_rows
allRows_selCol
pOfAll = allRows_selCol/numNAs
pOfAll
#books = books.append({'percentage_of_all':})

### Create Overview of Columns, Rows etc. for all categories

In [143]:
overview = pd.DataFrame()
for index in columns_sorted.filename:
    allTables = pd.DataFrame()
    print(index)
    inputdir = '../../src/data/schemafiltereddata/'+index+'/'
    files = [file for file in os.listdir(inputdir) if file.endswith('.json.gz')]
    #setcol = set(columns_sorted.loc[index]['listofcolumns'])
    for file in files:
        df = pd.read_json(os.path.join(inputdir+file), compression = 'gzip', lines=True)
        setcol = set(columns_sorted.loc[index]['listofcolumns'])
        numrows = df.shape[0]
        numcolumns = df.shape[1]
        numselcol = df[df.columns[df.columns.isin(setcol)]].shape[1]
        numNAs = list(df[df.columns[df.columns.isin(setcol)]].isnull().sum())
        list_col = list(df.columns[df.columns.isin(setcol)])
        allTables = allTables.append({'Class':index,'filename':file,'num_rows':numrows, 'num_columns': numcolumns, 'num_columns_selected':numselcol, 'numNAs': numNAs, 'columns':list_col}, ignore_index=True)
    overview = overview.append(allTables)
    display(overview)
        

Book


,Class,columns,filename,numNAs,num_columns,num_columns_selected,num_rows
0,Book,"[name, author, publisher, offers, bookedition]",Book_foldvaribooks.com_September2020.json.gz,"[1, 40, 157, 0, 138]",8.0,5.0,274.0
1,Book,"[name, author, publisher, isbn, datepublished,...",Book_virago.co.uk_September2020.json.gz,"[0, 2, 0, 0, 0, 1]",11.0,6.0,398.0
2,Book,"[name, author, publisher, offers, bookedition]",Book_borgantiquarian.com_September2020.json.gz,"[0, 0, 157, 0, 159]",8.0,5.0,248.0
3,Book,"[name, author, publisher, isbn, datepublished]",Book_downpour.com_September2020.json.gz,"[0, 13, 0, 7323, 22]",8.0,5.0,12194.0
4,Book,"[name, author, publisher, isbn, datepublished,...",Book_marasworld.com_September2020.json.gz,"[0, 0, 0, 0, 0, 0]",8.0,6.0,26.0
...,...,...,...,...,...,...,...
973,Book,"[name, publisher, isbn, bookformat, datepublis...",Book_benjamins.com_September2020.json.gz,"[0, 7, 756, 755, 62, 775, 8, 0]",14.0,8.0,1795.0
974,Book,"[name, author, publisher, isbn, datepublished,...",Book_nollegiu.com_September2020.json.gz,"[0, 77, 0, 36, 1465, 0, 561]",10.0,7.0,3175.0
975,Book,"[name, author, publisher, isbn, datepublished,...",Book_nosegraze.com_September2020.json.gz,"[0, 0, 0, 109, 0, 1, 1]",9.0,7.0,153.0
976,Book,"[name, author, publisher, isbn, datepublished,...",Book_hachettebooks.com_September2020.json.gz,"[0, 0, 0, 0, 0, 45, 1]",12.0,7.0,583.0


CreativeWork


,Class,columns,filename,numNAs,num_columns,num_columns_selected,num_rows
0,Book,"[name, author, publisher, offers, bookedition]",Book_foldvaribooks.com_September2020.json.gz,"[1, 40, 157, 0, 138]",8.0,5.0,274.0
1,Book,"[name, author, publisher, isbn, datepublished,...",Book_virago.co.uk_September2020.json.gz,"[0, 2, 0, 0, 0, 1]",11.0,6.0,398.0
2,Book,"[name, author, publisher, offers, bookedition]",Book_borgantiquarian.com_September2020.json.gz,"[0, 0, 157, 0, 159]",8.0,5.0,248.0
3,Book,"[name, author, publisher, isbn, datepublished]",Book_downpour.com_September2020.json.gz,"[0, 13, 0, 7323, 22]",8.0,5.0,12194.0
4,Book,"[name, author, publisher, isbn, datepublished,...",Book_marasworld.com_September2020.json.gz,"[0, 0, 0, 0, 0, 0]",8.0,6.0,26.0
...,...,...,...,...,...,...,...
5513,CreativeWork,"[headline, datepublished, datemodified, commen...",CreativeWork_kellyclaytonliving.com_September2...,"[0, 0, 0, 0]",6.0,4.0,15.0
5514,CreativeWork,"[headline, author, datemodified, datecreated, ...",CreativeWork_z125owners.com_September2020.json.gz,"[0, 0, 0, 0, 0]",8.0,5.0,108.0
5515,CreativeWork,"[headline, datepublished, datemodified, commen...",CreativeWork_cottages.com_September2020.json.gz,"[0, 0, 0, 0]",6.0,4.0,15.0
5516,CreativeWork,"[headline, author, datepublished, datemodified...",CreativeWork_preservationbuffaloniagara.org_Se...,"[1, 6, 0, 0, 1, 6, 1]",11.0,7.0,168.0


Event


...

,Class,columns,filename,numNAs,num_columns,num_columns_selected,num_rows
0,Book,"[name, author, publisher, offers, bookedition]",Book_foldvaribooks.com_September2020.json.gz,"[1, 40, 157, 0, 138]",8.0,5.0,274.0
1,Book,"[name, author, publisher, isbn, datepublished,...",Book_virago.co.uk_September2020.json.gz,"[0, 2, 0, 0, 0, 1]",11.0,6.0,398.0
2,Book,"[name, author, publisher, offers, bookedition]",Book_borgantiquarian.com_September2020.json.gz,"[0, 0, 157, 0, 159]",8.0,5.0,248.0
3,Book,"[name, author, publisher, isbn, datepublished]",Book_downpour.com_September2020.json.gz,"[0, 13, 0, 7323, 22]",8.0,5.0,12194.0
4,Book,"[name, author, publisher, isbn, datepublished,...",Book_marasworld.com_September2020.json.gz,"[0, 0, 0, 0, 0, 0]",8.0,6.0,26.0
...,...,...,...,...,...,...,...
7827,Recipe,"[name, recipeinstructions, author, recipeingre...",Recipe_cookingatclarktowers.com_September2020....,"[0, 0, 0, 0, 25, 0, 28, 30, 28]",12.0,9.0,41.0
7828,Recipe,"[name, recipeinstructions, author, recipeingre...",Recipe_mjandhungryman.com_September2020.json.gz,"[0, 0, 0, 0, 13, 0, 10, 13, 13, 16, 19, 19, 22...",19.0,16.0,42.0
7829,Recipe,"[name, recipeinstructions, recipeingredient, p...",Recipe_chefjanapinheiro.com_September2020.json.gz,"[0, 1, 0, 0, 0, 0, 11, 4, 15]",11.0,9.0,27.0
7830,Recipe,"[name, recipeinstructions, author, recipeingre...",Recipe_madcreationshub.com_September2020.json.gz,"[0, 0, 0, 0, 0, 0, 0, 4, 1, 0, 0, 3, 2, 0, 0, 0]",19.0,16.0,57.0


Restaurant


,Class,columns,filename,numNAs,num_columns,num_columns_selected,num_rows
0,Book,"[name, author, publisher, offers, bookedition]",Book_foldvaribooks.com_September2020.json.gz,"[1, 40, 157, 0, 138]",8.0,5.0,274.0
1,Book,"[name, author, publisher, isbn, datepublished,...",Book_virago.co.uk_September2020.json.gz,"[0, 2, 0, 0, 0, 1]",11.0,6.0,398.0
2,Book,"[name, author, publisher, offers, bookedition]",Book_borgantiquarian.com_September2020.json.gz,"[0, 0, 157, 0, 159]",8.0,5.0,248.0
3,Book,"[name, author, publisher, isbn, datepublished]",Book_downpour.com_September2020.json.gz,"[0, 13, 0, 7323, 22]",8.0,5.0,12194.0
4,Book,"[name, author, publisher, isbn, datepublished,...",Book_marasworld.com_September2020.json.gz,"[0, 0, 0, 0, 0, 0]",8.0,6.0,26.0
...,...,...,...,...,...,...,...
729,Restaurant,"[name, address, telephone, geo, servescuisine,...",Restaurant_sakura-teppan-yaki.com_September202...,"[0, 0, 0, 0, 0, 0, 0, 20, 0, 0]",13.0,10.0,28.0
730,Restaurant,"[name, address, telephone, servescuisine, aggr...",Restaurant_zabihah.com_September2020.json.gz,"[0, 0, 152, 95, 1151, 0, 0]",9.0,7.0,2528.0
731,Restaurant,"[name, address, telephone, geo, openinghourssp...",Restaurant_newks.com_September2020.json.gz,"[0, 0, 0, 0, 0, 0]",13.0,6.0,46.0
732,Restaurant,"[name, address, telephone, geo, servescuisine,...",Restaurant_le-sarrail.com_September2020.json.gz,"[0, 0, 0, 0, 3, 19, 0, 0, 0, 0]",13.0,10.0,28.0


TVEpisode


,Class,columns,filename,numNAs,num_columns,num_columns_selected,num_rows
0,Book,"[name, author, publisher, offers, bookedition]",Book_foldvaribooks.com_September2020.json.gz,"[1, 40, 157, 0, 138]",8.0,5.0,274.0
1,Book,"[name, author, publisher, isbn, datepublished,...",Book_virago.co.uk_September2020.json.gz,"[0, 2, 0, 0, 0, 1]",11.0,6.0,398.0
2,Book,"[name, author, publisher, offers, bookedition]",Book_borgantiquarian.com_September2020.json.gz,"[0, 0, 157, 0, 159]",8.0,5.0,248.0
3,Book,"[name, author, publisher, isbn, datepublished]",Book_downpour.com_September2020.json.gz,"[0, 13, 0, 7323, 22]",8.0,5.0,12194.0
4,Book,"[name, author, publisher, isbn, datepublished,...",Book_marasworld.com_September2020.json.gz,"[0, 0, 0, 0, 0, 0]",8.0,6.0,26.0
...,...,...,...,...,...,...,...
205,TVEpisode,"[name, episodenumber, partofseries]",TVEpisode_tvbuzer.com_September2020.json.gz,"[1, 27, 251]",9.0,3.0,1768.0
206,TVEpisode,"[name, episodenumber, partofseries]",TVEpisode_farsiland.com_September2020.json.gz,"[0, 0, 0]",13.0,3.0,53.0
207,TVEpisode,"[name, episodenumber]",TVEpisode_seriestowatch.com_September2020.json.gz,"[0, 0]",6.0,2.0,953.0
208,TVEpisode,"[name, episodenumber, partofseries]",TVEpisode_foodnetwork.com_September2020.json.gz,"[0, 0, 114]",7.0,3.0,431.0


In [157]:
overview_nas = pd.DataFrame()
nas = overview.filter(['numNAs'], axis=1)
#overview_nas = overview_nas.append[{'individual_NAs':nas}]
#overview_nas
nas = nas.explode('numNAs').rename(columns={'numNAs':'individual_NAs'})
nas

,individual_NAs
0,1
0,40
0,157
0,0
0,138
...,...
208,0
208,0
208,114
209,0


In [187]:
together = pd.concat([overview_detailed, nas], axis=1)
together

,Class,columns,filename,numNAs,num_columns,num_columns_selected,num_rows,individual_NAs
0,Book,name,Book_foldvaribooks.com_September2020.json.gz,"[1, 40, 157, 0, 138]",8.0,5.0,274.0,1
0,Book,author,Book_foldvaribooks.com_September2020.json.gz,"[1, 40, 157, 0, 138]",8.0,5.0,274.0,40
0,Book,publisher,Book_foldvaribooks.com_September2020.json.gz,"[1, 40, 157, 0, 138]",8.0,5.0,274.0,157
0,Book,offers,Book_foldvaribooks.com_September2020.json.gz,"[1, 40, 157, 0, 138]",8.0,5.0,274.0,0
0,Book,bookedition,Book_foldvaribooks.com_September2020.json.gz,"[1, 40, 157, 0, 138]",8.0,5.0,274.0,138
...,...,...,...,...,...,...,...,...
208,TVEpisode,name,TVEpisode_foodnetwork.com_September2020.json.gz,"[0, 0, 114]",7.0,3.0,431.0,0
208,TVEpisode,episodenumber,TVEpisode_foodnetwork.com_September2020.json.gz,"[0, 0, 114]",7.0,3.0,431.0,0
208,TVEpisode,partofseries,TVEpisode_foodnetwork.com_September2020.json.gz,"[0, 0, 114]",7.0,3.0,431.0,114
209,TVEpisode,name,TVEpisode_vavivov.com_September2020.json.gz,"[0, 0]",9.0,2.0,63.0,0


In [192]:
together = together.drop(columns = 'numNAs')
#together = together.rename(columns={'individual_NAs':'num_NAs'})
#together.reset_index()

In [156]:
overview_detailed = overview.explode('columns')
overview_detailed

,Class,columns,filename,numNAs,num_columns,num_columns_selected,num_rows
0,Book,name,Book_foldvaribooks.com_September2020.json.gz,"[1, 40, 157, 0, 138]",8.0,5.0,274.0
0,Book,author,Book_foldvaribooks.com_September2020.json.gz,"[1, 40, 157, 0, 138]",8.0,5.0,274.0
0,Book,publisher,Book_foldvaribooks.com_September2020.json.gz,"[1, 40, 157, 0, 138]",8.0,5.0,274.0
0,Book,offers,Book_foldvaribooks.com_September2020.json.gz,"[1, 40, 157, 0, 138]",8.0,5.0,274.0
0,Book,bookedition,Book_foldvaribooks.com_September2020.json.gz,"[1, 40, 157, 0, 138]",8.0,5.0,274.0
...,...,...,...,...,...,...,...
208,TVEpisode,name,TVEpisode_foodnetwork.com_September2020.json.gz,"[0, 0, 114]",7.0,3.0,431.0
208,TVEpisode,episodenumber,TVEpisode_foodnetwork.com_September2020.json.gz,"[0, 0, 114]",7.0,3.0,431.0
208,TVEpisode,partofseries,TVEpisode_foodnetwork.com_September2020.json.gz,"[0, 0, 114]",7.0,3.0,431.0
209,TVEpisode,name,TVEpisode_vavivov.com_September2020.json.gz,"[0, 0]",9.0,2.0,63.0


In [193]:
overview.to_csv('../../src/data/data/CSV_files/col_row_statistics_all_files.csv')
together.to_csv('../../src/data/data/CSV_files/detailed_columnStatistics_all_files.csv')

In [165]:
overview

,Class,columns,filename,numNAs,num_columns,num_columns_selected,num_rows
0,Book,"[name, author, publisher, offers, bookedition]",Book_foldvaribooks.com_September2020.json.gz,"[1, 40, 157, 0, 138]",8.0,5.0,274.0
1,Book,"[name, author, publisher, isbn, datepublished,...",Book_virago.co.uk_September2020.json.gz,"[0, 2, 0, 0, 0, 1]",11.0,6.0,398.0
2,Book,"[name, author, publisher, offers, bookedition]",Book_borgantiquarian.com_September2020.json.gz,"[0, 0, 157, 0, 159]",8.0,5.0,248.0
3,Book,"[name, author, publisher, isbn, datepublished]",Book_downpour.com_September2020.json.gz,"[0, 13, 0, 7323, 22]",8.0,5.0,12194.0
4,Book,"[name, author, publisher, isbn, datepublished,...",Book_marasworld.com_September2020.json.gz,"[0, 0, 0, 0, 0, 0]",8.0,6.0,26.0
...,...,...,...,...,...,...,...
205,TVEpisode,"[name, episodenumber, partofseries]",TVEpisode_tvbuzer.com_September2020.json.gz,"[1, 27, 251]",9.0,3.0,1768.0
206,TVEpisode,"[name, episodenumber, partofseries]",TVEpisode_farsiland.com_September2020.json.gz,"[0, 0, 0]",13.0,3.0,53.0
207,TVEpisode,"[name, episodenumber]",TVEpisode_seriestowatch.com_September2020.json.gz,"[0, 0]",6.0,2.0,953.0
208,TVEpisode,"[name, episodenumber, partofseries]",TVEpisode_foodnetwork.com_September2020.json.gz,"[0, 0, 114]",7.0,3.0,431.0


In [176]:
#together = together.reset_index()
together.drop(columns=['level_0'])
together

,level_0,index,Class,columns,filename,num_columns,num_columns_selected,num_rows,num_NAs
0,0,0,Book,name,Book_foldvaribooks.com_September2020.json.gz,8.0,5.0,274.0,1
1,1,0,Book,author,Book_foldvaribooks.com_September2020.json.gz,8.0,5.0,274.0,40
2,2,0,Book,publisher,Book_foldvaribooks.com_September2020.json.gz,8.0,5.0,274.0,157
3,3,0,Book,offers,Book_foldvaribooks.com_September2020.json.gz,8.0,5.0,274.0,0
4,4,0,Book,bookedition,Book_foldvaribooks.com_September2020.json.gz,8.0,5.0,274.0,138
...,...,...,...,...,...,...,...,...,...
459242,459242,208,TVEpisode,name,TVEpisode_foodnetwork.com_September2020.json.gz,7.0,3.0,431.0,0
459243,459243,208,TVEpisode,episodenumber,TVEpisode_foodnetwork.com_September2020.json.gz,7.0,3.0,431.0,0
459244,459244,208,TVEpisode,partofseries,TVEpisode_foodnetwork.com_September2020.json.gz,7.0,3.0,431.0,114
459245,459245,209,TVEpisode,name,TVEpisode_vavivov.com_September2020.json.gz,9.0,2.0,63.0,0


In [178]:
col_less_than_10 = together[together['num_rows']<=10]
col_less_than_10

,level_0,index,Class,columns,filename,num_columns,num_columns_selected,num_rows,num_NAs
18057,18057,1840,CreativeWork,headline,CreativeWork_rayscatering.com_September2020.js...,8.0,4.0,4.0,0
18058,18058,1840,CreativeWork,datepublished,CreativeWork_rayscatering.com_September2020.js...,8.0,4.0,4.0,0
18059,18059,1840,CreativeWork,datemodified,CreativeWork_rayscatering.com_September2020.js...,8.0,4.0,4.0,0
18060,18060,1840,CreativeWork,commentcount,CreativeWork_rayscatering.com_September2020.js...,8.0,4.0,4.0,0
18207,18207,1864,CreativeWork,headline,CreativeWork_yuspecialtycafe.com_September2020...,6.0,3.0,10.0,0
...,...,...,...,...,...,...,...,...,...
152507,152507,12658,MusicRecording,byartist,MusicRecording_shipyardwreckmusic.com_Septembe...,5.0,3.0,9.0,1
152511,152511,12660,MusicRecording,name,MusicRecording_lynettediaz.com_September2020.j...,6.0,4.0,7.0,0
152512,152512,12660,MusicRecording,duration,MusicRecording_lynettediaz.com_September2020.j...,6.0,4.0,7.0,0
152513,152513,12660,MusicRecording,byartist,MusicRecording_lynettediaz.com_September2020.j...,6.0,4.0,7.0,3


In [179]:
table_less_than_10 = overview[overview['num_rows']<=10]
table_less_than_10

,Class,columns,filename,numNAs,num_columns,num_columns_selected,num_rows
1840,CreativeWork,"[headline, datepublished, datemodified, commen...",CreativeWork_rayscatering.com_September2020.js...,"[0, 0, 0, 0]",8.0,4.0,4.0
1864,CreativeWork,"[headline, author, interactiontype]",CreativeWork_yuspecialtycafe.com_September2020...,"[0, 0, 0]",6.0,3.0,10.0
1871,CreativeWork,"[headline, author, interactiontype]",CreativeWork_deskilate.com_September2020.json.gz,"[0, 0, 0]",6.0,3.0,4.0
1872,CreativeWork,"[datepublished, name, genre]",CreativeWork_tresfaros.com_September2020.json.gz,"[0, 0, 0]",6.0,3.0,4.0
1880,CreativeWork,"[headline, datepublished, datemodified, commen...",CreativeWork_actionpestexterminating.com_Septe...,"[0, 0, 0, 0]",8.0,4.0,3.0
...,...,...,...,...,...,...,...
12653,MusicRecording,"[name, duration, byartist]",MusicRecording_smokinonplanes.com_September202...,"[0, 0, 0]",5.0,3.0,9.0
12654,MusicRecording,"[name, duration, byartist, inalbum]",MusicRecording_xavierasali.com_September2020.j...,"[0, 0, 0, 2]",6.0,4.0,9.0
12656,MusicRecording,"[name, duration, byartist, inalbum]",MusicRecording_indigenousrocks.com_September20...,"[0, 0, 1, 0]",6.0,4.0,5.0
12658,MusicRecording,"[name, duration, byartist]",MusicRecording_shipyardwreckmusic.com_Septembe...,"[0, 0, 1]",5.0,3.0,9.0


In [204]:
table_10_rows = overview[overview['num_rows']>10]
table_10_rows

,Class,columns,filename,numNAs,num_columns,num_columns_selected,num_rows
0,Book,"[name, author, publisher, offers, bookedition]",Book_foldvaribooks.com_September2020.json.gz,"[1, 40, 157, 0, 138]",8.0,5.0,274.0
1,Book,"[name, author, publisher, isbn, datepublished,...",Book_virago.co.uk_September2020.json.gz,"[0, 2, 0, 0, 0, 1]",11.0,6.0,398.0
2,Book,"[name, author, publisher, offers, bookedition]",Book_borgantiquarian.com_September2020.json.gz,"[0, 0, 157, 0, 159]",8.0,5.0,248.0
3,Book,"[name, author, publisher, isbn, datepublished]",Book_downpour.com_September2020.json.gz,"[0, 13, 0, 7323, 22]",8.0,5.0,12194.0
4,Book,"[name, author, publisher, isbn, datepublished,...",Book_marasworld.com_September2020.json.gz,"[0, 0, 0, 0, 0, 0]",8.0,6.0,26.0
...,...,...,...,...,...,...,...
205,TVEpisode,"[name, episodenumber, partofseries]",TVEpisode_tvbuzer.com_September2020.json.gz,"[1, 27, 251]",9.0,3.0,1768.0
206,TVEpisode,"[name, episodenumber, partofseries]",TVEpisode_farsiland.com_September2020.json.gz,"[0, 0, 0]",13.0,3.0,53.0
207,TVEpisode,"[name, episodenumber]",TVEpisode_seriestowatch.com_September2020.json.gz,"[0, 0]",6.0,2.0,953.0
208,TVEpisode,"[name, episodenumber, partofseries]",TVEpisode_foodnetwork.com_September2020.json.gz,"[0, 0, 114]",7.0,3.0,431.0


In [240]:
#NA count for entire table
overview_2 = pd.DataFrame()
for index in columns_sorted.filename:
    allTables_2 = pd.DataFrame()
    print(index)
    inputdir = '../../src/data/schemafiltereddata/'+index+'/'
    files = [file for file in os.listdir(inputdir) if file.endswith('.json.gz')]
    #setcol = set(columns_sorted.loc[index]['listofcolumns'])
    for file in files:
        df = pd.read_json(os.path.join(inputdir+file), compression = 'gzip', lines=True)
        setcol = set(columns_sorted.loc[index]['listofcolumns'])
        numrows = df.shape[0]
        numcolumns = df.shape[1]
        numselcol = df[df.columns[df.columns.isin(setcol)]].shape[1]
        numNAs = df[df.columns[df.columns.isin(setcol)]].isnull().sum().sum()
        list_col = list(df.columns[df.columns.isin(setcol)])
        allTables_2 = allTables_2.append({'Class':index,'filename':file,'num_rows':numrows, 'num_columns': numcolumns, 'num_columns_selected':numselcol, 'numNAs': numNAs, 'columns':list_col}, ignore_index=True)
    overview_2 = overview_2.append(allTables_2)
    display(overview_2)

Book


,Class,columns,filename,numNAs,num_columns,num_columns_selected,num_rows
0,Book,"[name, author, publisher, offers, bookedition]",Book_foldvaribooks.com_September2020.json.gz,336.0,8.0,5.0,274.0
1,Book,"[name, author, publisher, isbn, datepublished,...",Book_virago.co.uk_September2020.json.gz,3.0,11.0,6.0,398.0
2,Book,"[name, author, publisher, offers, bookedition]",Book_borgantiquarian.com_September2020.json.gz,316.0,8.0,5.0,248.0
3,Book,"[name, author, publisher, isbn, datepublished]",Book_downpour.com_September2020.json.gz,7358.0,8.0,5.0,12194.0
4,Book,"[name, author, publisher, isbn, datepublished,...",Book_marasworld.com_September2020.json.gz,0.0,8.0,6.0,26.0
...,...,...,...,...,...,...,...
973,Book,"[name, publisher, isbn, bookformat, datepublis...",Book_benjamins.com_September2020.json.gz,2363.0,14.0,8.0,1795.0
974,Book,"[name, author, publisher, isbn, datepublished,...",Book_nollegiu.com_September2020.json.gz,2139.0,10.0,7.0,3175.0
975,Book,"[name, author, publisher, isbn, datepublished,...",Book_nosegraze.com_September2020.json.gz,111.0,9.0,7.0,153.0
976,Book,"[name, author, publisher, isbn, datepublished,...",Book_hachettebooks.com_September2020.json.gz,46.0,12.0,7.0,583.0


CreativeWork


,Class,columns,filename,numNAs,num_columns,num_columns_selected,num_rows
0,Book,"[name, author, publisher, offers, bookedition]",Book_foldvaribooks.com_September2020.json.gz,336.0,8.0,5.0,274.0
1,Book,"[name, author, publisher, isbn, datepublished,...",Book_virago.co.uk_September2020.json.gz,3.0,11.0,6.0,398.0
2,Book,"[name, author, publisher, offers, bookedition]",Book_borgantiquarian.com_September2020.json.gz,316.0,8.0,5.0,248.0
3,Book,"[name, author, publisher, isbn, datepublished]",Book_downpour.com_September2020.json.gz,7358.0,8.0,5.0,12194.0
4,Book,"[name, author, publisher, isbn, datepublished,...",Book_marasworld.com_September2020.json.gz,0.0,8.0,6.0,26.0
...,...,...,...,...,...,...,...
5513,CreativeWork,"[headline, datepublished, datemodified, commen...",CreativeWork_kellyclaytonliving.com_September2...,0.0,6.0,4.0,15.0
5514,CreativeWork,"[headline, author, datemodified, datecreated, ...",CreativeWork_z125owners.com_September2020.json.gz,0.0,8.0,5.0,108.0
5515,CreativeWork,"[headline, datepublished, datemodified, commen...",CreativeWork_cottages.com_September2020.json.gz,0.0,6.0,4.0,15.0
5516,CreativeWork,"[headline, author, datepublished, datemodified...",CreativeWork_preservationbuffaloniagara.org_Se...,15.0,11.0,7.0,168.0


Event


...

,Class,columns,filename,numNAs,num_columns,num_columns_selected,num_rows
0,Book,"[name, author, publisher, offers, bookedition]",Book_foldvaribooks.com_September2020.json.gz,336.0,8.0,5.0,274.0
1,Book,"[name, author, publisher, isbn, datepublished,...",Book_virago.co.uk_September2020.json.gz,3.0,11.0,6.0,398.0
2,Book,"[name, author, publisher, offers, bookedition]",Book_borgantiquarian.com_September2020.json.gz,316.0,8.0,5.0,248.0
3,Book,"[name, author, publisher, isbn, datepublished]",Book_downpour.com_September2020.json.gz,7358.0,8.0,5.0,12194.0
4,Book,"[name, author, publisher, isbn, datepublished,...",Book_marasworld.com_September2020.json.gz,0.0,8.0,6.0,26.0
...,...,...,...,...,...,...,...
7827,Recipe,"[name, recipeinstructions, author, recipeingre...",Recipe_cookingatclarktowers.com_September2020....,111.0,12.0,9.0,41.0
7828,Recipe,"[name, recipeinstructions, author, recipeingre...",Recipe_mjandhungryman.com_September2020.json.gz,135.0,19.0,16.0,42.0
7829,Recipe,"[name, recipeinstructions, recipeingredient, p...",Recipe_chefjanapinheiro.com_September2020.json.gz,31.0,11.0,9.0,27.0
7830,Recipe,"[name, recipeinstructions, author, recipeingre...",Recipe_madcreationshub.com_September2020.json.gz,10.0,19.0,16.0,57.0


Restaurant


,Class,columns,filename,numNAs,num_columns,num_columns_selected,num_rows
0,Book,"[name, author, publisher, offers, bookedition]",Book_foldvaribooks.com_September2020.json.gz,336.0,8.0,5.0,274.0
1,Book,"[name, author, publisher, isbn, datepublished,...",Book_virago.co.uk_September2020.json.gz,3.0,11.0,6.0,398.0
2,Book,"[name, author, publisher, offers, bookedition]",Book_borgantiquarian.com_September2020.json.gz,316.0,8.0,5.0,248.0
3,Book,"[name, author, publisher, isbn, datepublished]",Book_downpour.com_September2020.json.gz,7358.0,8.0,5.0,12194.0
4,Book,"[name, author, publisher, isbn, datepublished,...",Book_marasworld.com_September2020.json.gz,0.0,8.0,6.0,26.0
...,...,...,...,...,...,...,...
729,Restaurant,"[name, address, telephone, geo, servescuisine,...",Restaurant_sakura-teppan-yaki.com_September202...,20.0,13.0,10.0,28.0
730,Restaurant,"[name, address, telephone, servescuisine, aggr...",Restaurant_zabihah.com_September2020.json.gz,1398.0,9.0,7.0,2528.0
731,Restaurant,"[name, address, telephone, geo, openinghourssp...",Restaurant_newks.com_September2020.json.gz,0.0,13.0,6.0,46.0
732,Restaurant,"[name, address, telephone, geo, servescuisine,...",Restaurant_le-sarrail.com_September2020.json.gz,22.0,13.0,10.0,28.0


TVEpisode


,Class,columns,filename,numNAs,num_columns,num_columns_selected,num_rows
0,Book,"[name, author, publisher, offers, bookedition]",Book_foldvaribooks.com_September2020.json.gz,336.0,8.0,5.0,274.0
1,Book,"[name, author, publisher, isbn, datepublished,...",Book_virago.co.uk_September2020.json.gz,3.0,11.0,6.0,398.0
2,Book,"[name, author, publisher, offers, bookedition]",Book_borgantiquarian.com_September2020.json.gz,316.0,8.0,5.0,248.0
3,Book,"[name, author, publisher, isbn, datepublished]",Book_downpour.com_September2020.json.gz,7358.0,8.0,5.0,12194.0
4,Book,"[name, author, publisher, isbn, datepublished,...",Book_marasworld.com_September2020.json.gz,0.0,8.0,6.0,26.0
...,...,...,...,...,...,...,...
205,TVEpisode,"[name, episodenumber, partofseries]",TVEpisode_tvbuzer.com_September2020.json.gz,279.0,9.0,3.0,1768.0
206,TVEpisode,"[name, episodenumber, partofseries]",TVEpisode_farsiland.com_September2020.json.gz,0.0,13.0,3.0,53.0
207,TVEpisode,"[name, episodenumber]",TVEpisode_seriestowatch.com_September2020.json.gz,0.0,6.0,2.0,953.0
208,TVEpisode,"[name, episodenumber, partofseries]",TVEpisode_foodnetwork.com_September2020.json.gz,114.0,7.0,3.0,431.0


In [264]:
#overview_2.explode('columns').reset_index()
#overview_2 = overview_2.rename(columns={'numNAs':'count_NAs'})
#overview_2.reset_index()
overview_2 = overview_2.explode('columns')
overview_2

,Class,columns,filename,count_NAs,num_columns,num_columns_selected,num_rows
0,Book,name,Book_foldvaribooks.com_September2020.json.gz,336.0,8.0,5.0,274.0
0,Book,author,Book_foldvaribooks.com_September2020.json.gz,336.0,8.0,5.0,274.0
0,Book,publisher,Book_foldvaribooks.com_September2020.json.gz,336.0,8.0,5.0,274.0
0,Book,offers,Book_foldvaribooks.com_September2020.json.gz,336.0,8.0,5.0,274.0
0,Book,bookedition,Book_foldvaribooks.com_September2020.json.gz,336.0,8.0,5.0,274.0
...,...,...,...,...,...,...,...
208,TVEpisode,name,TVEpisode_foodnetwork.com_September2020.json.gz,114.0,7.0,3.0,431.0
208,TVEpisode,episodenumber,TVEpisode_foodnetwork.com_September2020.json.gz,114.0,7.0,3.0,431.0
208,TVEpisode,partofseries,TVEpisode_foodnetwork.com_September2020.json.gz,114.0,7.0,3.0,431.0
209,TVEpisode,name,TVEpisode_vavivov.com_September2020.json.gz,0.0,9.0,2.0,63.0


In [249]:
together.reset_index()

,index,Class,columns,filename,num_columns,num_columns_selected,num_rows,num_NAs,percentage_NAs
0,0,Book,name,Book_foldvaribooks.com_September2020.json.gz,8.0,5.0,274.0,1,0.00365
1,0,Book,author,Book_foldvaribooks.com_September2020.json.gz,8.0,5.0,274.0,40,0.145985
2,0,Book,publisher,Book_foldvaribooks.com_September2020.json.gz,8.0,5.0,274.0,157,0.572993
3,0,Book,offers,Book_foldvaribooks.com_September2020.json.gz,8.0,5.0,274.0,0,0.0
4,0,Book,bookedition,Book_foldvaribooks.com_September2020.json.gz,8.0,5.0,274.0,138,0.50365
...,...,...,...,...,...,...,...,...,...
459242,208,TVEpisode,name,TVEpisode_foodnetwork.com_September2020.json.gz,7.0,3.0,431.0,0,0.0
459243,208,TVEpisode,episodenumber,TVEpisode_foodnetwork.com_September2020.json.gz,7.0,3.0,431.0,0,0.0
459244,208,TVEpisode,partofseries,TVEpisode_foodnetwork.com_September2020.json.gz,7.0,3.0,431.0,114,0.264501
459245,209,TVEpisode,name,TVEpisode_vavivov.com_September2020.json.gz,9.0,2.0,63.0,0,0.0


In [269]:
#together['sum_NA_tables'] = overview_2['count_NAs']
#together = together.rename(columns={'percentage_NAs':'percentage_col_NA'})
total_NA_p = together.sum_NA_tables/(together.num_columns_selected*together.num_rows)
together['percentage_table_NA'] = total_NA_p
together

,Class,columns,filename,num_columns,num_columns_selected,num_rows,num_NAs,percentage_col_NA,sum_NA_tables,percentage_table_NA
0,Book,name,Book_foldvaribooks.com_September2020.json.gz,8.0,5.0,274.0,1,0.00365,336.0,0.245255
0,Book,author,Book_foldvaribooks.com_September2020.json.gz,8.0,5.0,274.0,40,0.145985,336.0,0.245255
0,Book,publisher,Book_foldvaribooks.com_September2020.json.gz,8.0,5.0,274.0,157,0.572993,336.0,0.245255
0,Book,offers,Book_foldvaribooks.com_September2020.json.gz,8.0,5.0,274.0,0,0.0,336.0,0.245255
0,Book,bookedition,Book_foldvaribooks.com_September2020.json.gz,8.0,5.0,274.0,138,0.50365,336.0,0.245255
...,...,...,...,...,...,...,...,...,...,...
208,TVEpisode,name,TVEpisode_foodnetwork.com_September2020.json.gz,7.0,3.0,431.0,0,0.0,114.0,0.088167
208,TVEpisode,episodenumber,TVEpisode_foodnetwork.com_September2020.json.gz,7.0,3.0,431.0,0,0.0,114.0,0.088167
208,TVEpisode,partofseries,TVEpisode_foodnetwork.com_September2020.json.gz,7.0,3.0,431.0,114,0.264501,114.0,0.088167
209,TVEpisode,name,TVEpisode_vavivov.com_September2020.json.gz,9.0,2.0,63.0,0,0.0,0.0,0.000000


In [270]:
together.to_csv('../../src/data/data/CSV_files/detailed_columnStatistics_all_files.csv')

In [ ]:
NA_rows = together.num_NAs/together.num_rows
together['percentage_NAs'] = NA_rows
together

In [271]:
together_small_NA = together[(together['percentage_col_NA']<0.25) & (together['percentage_table_NA']<0.25)]
together_small_NA

,Class,columns,filename,num_columns,num_columns_selected,num_rows,num_NAs,percentage_col_NA,sum_NA_tables,percentage_table_NA
0,Book,name,Book_foldvaribooks.com_September2020.json.gz,8.0,5.0,274.0,1,0.00365,336.0,0.245255
0,Book,author,Book_foldvaribooks.com_September2020.json.gz,8.0,5.0,274.0,40,0.145985,336.0,0.245255
0,Book,offers,Book_foldvaribooks.com_September2020.json.gz,8.0,5.0,274.0,0,0.0,336.0,0.245255
1,Book,name,Book_virago.co.uk_September2020.json.gz,11.0,6.0,398.0,0,0.0,3.0,0.001256
1,Book,author,Book_virago.co.uk_September2020.json.gz,11.0,6.0,398.0,2,0.005025,3.0,0.001256
...,...,...,...,...,...,...,...,...,...,...
207,TVEpisode,episodenumber,TVEpisode_seriestowatch.com_September2020.json.gz,6.0,2.0,953.0,0,0.0,0.0,0.000000
208,TVEpisode,name,TVEpisode_foodnetwork.com_September2020.json.gz,7.0,3.0,431.0,0,0.0,114.0,0.088167
208,TVEpisode,episodenumber,TVEpisode_foodnetwork.com_September2020.json.gz,7.0,3.0,431.0,0,0.0,114.0,0.088167
209,TVEpisode,name,TVEpisode_vavivov.com_September2020.json.gz,9.0,2.0,63.0,0,0.0,0.0,0.000000


In [272]:
together.describe()

,num_columns,num_columns_selected,num_rows,sum_NA_tables,percentage_table_NA
count,459247.000000,459247.000000,459247.000000,459247.000000,459247.000000
mean,10.765004,6.976440,303.832312,220.986014,0.087485
std,3.687601,3.455867,1307.401293,1102.025216,0.103543
min,5.000000,2.000000,3.000000,0.000000,0.000000
25%,9.000000,5.000000,31.000000,1.000000,0.000356
50%,10.000000,6.000000,66.000000,20.000000,0.050441
75%,12.000000,8.000000,210.000000,104.000000,0.138889
max,35.000000,24.000000,222817.000000,47753.000000,0.818182


In [290]:
group = together.groupby(['filename']).agg({'num_rows':'mean', 'sum_NA_tables':'mean', 'percentage_table_NA':'mean'})
group

,num_rows,sum_NA_tables,percentage_table_NA
filename,,,
Book_12min.com_September2020.json.gz,366.0,166.0,0.090710
Book_19shuwu.com_September2020.json.gz,250.0,0.0,0.000000
Book_1carpetcleaning.co.uk_September2020.json.gz,1004.0,0.0,0.000000
Book_2014brazil.co.uk_September2020.json.gz,387.0,0.0,0.000000
Book_24symbols.com_September2020.json.gz,5171.0,17.0,0.000658
...,...,...,...
TVEpisode_yesmoviess.com_September2020.json.gz,37.0,0.0,0.000000
TVEpisode_yidio.com_September2020.json.gz,12606.0,8590.0,0.227141
TVEpisode_zazangels.com_September2020.json.gz,38.0,0.0,0.000000


In [294]:
relevant = group[(group['num_rows']>10) & (group['percentage_table_NA']<0.20)]
relevant

,num_rows,sum_NA_tables,percentage_table_NA
filename,,,
Book_12min.com_September2020.json.gz,366.0,166.0,0.090710
Book_19shuwu.com_September2020.json.gz,250.0,0.0,0.000000
Book_1carpetcleaning.co.uk_September2020.json.gz,1004.0,0.0,0.000000
Book_2014brazil.co.uk_September2020.json.gz,387.0,0.0,0.000000
Book_24symbols.com_September2020.json.gz,5171.0,17.0,0.000658
...,...,...,...
TVEpisode_yara-online.org_September2020.json.gz,40.0,0.0,0.000000
TVEpisode_yesmoviess.com_September2020.json.gz,37.0,0.0,0.000000
TVEpisode_zazangels.com_September2020.json.gz,38.0,0.0,0.000000


In [304]:
small = relevant[relevant['num_rows']<=40]
small

,num_rows,sum_NA_tables,percentage_table_NA
filename,,,
Book_aardvark-books.com_September2020.json.gz,26.0,5.0,0.027473
Book_americanreadingathome.com_September2020.json.gz,33.0,3.0,0.018182
Book_barmatrioshka.com_September2020.json.gz,35.0,0.0,0.000000
Book_bibliobeat.com_September2020.json.gz,22.0,0.0,0.000000
Book_bookapy.com_September2020.json.gz,26.0,11.0,0.052885
...,...,...,...
TVEpisode_wussmovie.com_September2020.json.gz,40.0,0.0,0.000000
TVEpisode_yara-online.org_September2020.json.gz,40.0,0.0,0.000000
TVEpisode_yesmoviess.com_September2020.json.gz,37.0,0.0,0.000000


In [305]:
medium = relevant[(relevant['num_rows']>40) & (relevant['num_rows']<=200)]
medium

,num_rows,sum_NA_tables,percentage_table_NA
filename,,,
Book_addicusbooks.com_September2020.json.gz,89.0,46.0,0.073836
Book_alexanderstreet.com_September2020.json.gz,197.0,146.0,0.148223
Book_amle.org_September2020.json.gz,41.0,16.0,0.065041
Book_artisanbooksandbindery.com_September2020.json.gz,199.0,182.0,0.182915
Book_auxlettresdemonmoulin.com_September2020.json.gz,141.0,90.0,0.106383
...,...,...,...
TVEpisode_wireless-satellite-internet.com_September2020.json.gz,54.0,0.0,0.000000
TVEpisode_ww2d.org_September2020.json.gz,54.0,0.0,0.000000
TVEpisode_wybrzezak.org_September2020.json.gz,50.0,0.0,0.000000


In [301]:
large = relevant[relevant['num_rows']>200]
large

,num_rows,sum_NA_tables,percentage_table_NA
filename,,,
Book_12min.com_September2020.json.gz,366.0,166.0,0.090710
Book_19shuwu.com_September2020.json.gz,250.0,0.0,0.000000
Book_1carpetcleaning.co.uk_September2020.json.gz,1004.0,0.0,0.000000
Book_2014brazil.co.uk_September2020.json.gz,387.0,0.0,0.000000
Book_24symbols.com_September2020.json.gz,5171.0,17.0,0.000658
...,...,...,...
TVEpisode_usanetwork.com_September2020.json.gz,306.0,0.0,0.000000
TVEpisode_vh1.com_September2020.json.gz,1448.0,8.0,0.001842
TVEpisode_wikidex.net_September2020.json.gz,504.0,34.0,0.033730


In [ ]:
#create folder with small/medium/large files
for index in columns_sorted.filename:
    print(index)
    inputdir = '../../src/data/schemafiltereddata/'+index+'/'
    files = [file for file in os.listdir(inputdir) if file.endswith('.json.gz')]
    #setcol = set(columns_sorted.loc[index]['listofcolumns'])
    for file in large.filename:
       df.to_json(os.path.join(outputdir+table), orient = 'records', compression = 'gzip', lines = True)
    overview_2 = overview_2.append(allTables_2)
    display(overview_2)

In [ ]:
#function to count rows and move tables that have less than 10 rows
def count_move_small_tables(table):
    df = pd.read_json(os.path.join(inputdir+table), compression='gzip', lines=True)
    if len(df)<=10:
        df.to_json(os.path.join(outputdir+table), orient = 'records', compression = 'gzip', lines = True)
        #os.remove(os.path.join(output, '{}'.format(file)))
        print(len(df))
    elif df[df[col for col in setcol].isna()]:
        
        #df.to_json(os.path.join(outputdir+table), orient = 'records', compression = 'gzip', lines = True)
        #os.remove(os.path.join(output, '{}'.format(file)))
        return
    else:
        row_df = pd.DataFrame ()
        row_df = row_df.append({'file': table, 'length': len(df)}, ignore_index=True)
    print(table, len(files))
    print(row_df)
    #print(row_df)

In [ ]:
#files = [file for file in os.listdir(inputdir) if file.endswith('.json.gz')]
#run function for all classes & tables with multiprocessing
#row_df = pd.DataFrame ()
if __name__ == '__main__':
    for index in columns_sorted.filename:
        print(index)
        setcol = set(columns_sorted.loc[index]['listofcolumns'])
        print(setcol)
        inputdir = '../../src/data/schemafiltereddata/'+index+'/'
        outputdir = '../../src/data/schemafiltereddata/'+index+'/under_10_rows_and_NAs/'
        if not os.path.exists(outputdir):
            os.makedirs(outputdir)
        p = Pool(100)
        for _ in tqdm(p.imap_unordered(count_move_small_tables, [file for file in os.listdir(inputdir) if file.endswith('.json.gz')]), total=len([file for file in os.listdir(inputdir) if file.endswith('.json.gz')])):
            pass
        p.map(count_move_small_tables, [file for file in os.listdir(inputdir) if file.endswith('.json.gz')])
        p.close()
        #print(len([file for file in os.listdir(outputdir) if file.endswith('.json.gz')]))
        #new_df = pd.DataFrame ()
        #new_df = new_df.append({'file': row_df['filename'.groupby(index))], 'mean_length': row_df.mean(len(df))}, ignore_index=True)
        p.join()

In [ ]:
print(row_df)

In [ ]:
row_df.mean(axis = 1).mean()

In [ ]:
num_tables=[]
for index in columns_sorted.filename:
    print(index)
    outputdir = '../../src/data/schemafiltereddata/'+index+'/under_10_rows_and_NAs/'
    print(len([file for file in os.listdir(outputdir) if file.endswith('.json.gz')]))
    num_tables.append(len([file for file in os.listdir(outputdir) if file.endswith('.json.gz')]))
len(num_tables)

In [ ]:
test_df = pd.read_json('/work-ceph/bizer-tp2021/data_integration_using_deep_learning/src/data/schemafiltereddata/CreativeWork/under_10_rows/CreativeWork_1daywebs.com_September2020.json.gz', compression='gzip', lines=True)
test_df

In [ ]:
overview = pd.read_csv(path+'/filestats_over_all_columns.csv')
overview

In [ ]:
print(overview[(overview['num_rows']<=10) & (overview['Class']=='CreativeWork')])